In [1]:
import docker
client = docker.from_env()

In [6]:
client.images.list()

[<Image: 'python_trans_conf:6'>,
 <Image: 'ad159:latest'>,
 <Image: 'nikromanov/solana-velas:1.5.0'>,
 <Image: 'trans_simul:0.2'>,
 <Image: 'solana:142'>,
 <Image: 'python:3.7'>,
 <Image: 'ubu_config:latest'>,
 <Image: 'ubuntu:latest'>]

In [5]:
client.containers.run('python_trans_conf:6', command="python velas-ss/transaction_sender.py --tps 100 --s 2",
                      volumes={'/Users/korg/PycharmProjects/velas-ss/tools/': {'bind': '/velas-ss/', 'mode': 'rw'},
                              '/Users/korg/PycharmProjects/velas-ss/my_vol/': {'bind': '/mnt/logs', 'mode': 'rw'}})

b''

In [9]:
solanka = client.containers.run('nikromanov/solana-velas:1.5.0', 
                      command='''bash -c 'sleep 3 && ./multinode-demo/setup.sh &&
                      nohup bash -c "./multinode-demo/bootstrap-validator.sh --enable-rpc-transaction-history --log /mnt/logs/solana_genesis_node.txt &" &&
                      ./multinode-demo/faucet.sh' ''',
                      volumes={'/Users/korg/PycharmProjects/velas-ss/my_vol/': {'bind': '/mnt/logs', 'mode': 'rw'}},
                      detach=True
                     )

In [16]:
!pip install pyyaml

In [17]:
import yaml

with open('docker-multinode/docker-stack.yml') as f:
    templates = yaml.safe_load(f)

print(templates)

{'version': '3.8', 'services': {'config_generator': {'image': 'python:3.8-slim', 'command': 'bash -c "python config_generator.py --output /solana/config/config.toml"', 'volumes': ['/mnt/nfs_share/solana/ad159v2/config:/solana/config', '../configs:/config_generator/'], 'working_dir': '/config_generator', 'user': '10000:2001', 'deploy': {'placement': {'constraints': ['node.role==manager']}}}, 'genesis_node': {'image': 'nikromanov/solana-velas:1.5.0', 'depends_on': ['config_generator'], 'command': 'bash -c \'sleep 3 && ./multinode-demo/setup.sh && nohup bash -c "./multinode-demo/bootstrap-validator.sh --enable-rpc-transaction-history --gossip-host genesis_node --log /mnt/logs/solana_genesis_node.txt &" && ./multinode-demo/faucet.sh\'', 'volumes': ['/mnt/nfs_share/solana/ad159v2/logs:/mnt/logs', '/mnt/nfs_share/solana/ad159v2/config:/solana/config'], 'environment': ['TOML_CONFIG=/solana/config/config.toml'], 'networks': ['solana_net']}, 'validator': {'image': 'nikromanov/solana-velas:1.5.0

In [18]:
import subprocess

In [21]:
subprocess.Popen(['ls', '-l'], stdout=subprocess.PIPE, stderr=subprocess.PIPE).communicate()

(b'total 112\n-rw-r--r--  1 korg  staff    174 Jan 12 23:56 Dockerfile\n-rw-r--r--  1 korg  staff    138 Dec 23 22:07 Pipfile\n-rw-r--r--  1 korg  staff    453 Dec 23 22:07 Pipfile.lock\n-rw-r--r--  1 korg  staff    840 Jan 12 15:16 README.md\ndrwxr-xr-x  5 korg  staff    160 Dec 23 22:07 configs\ndrwxr-xr-x  7 korg  staff    224 Dec 23 22:08 docker-multinode\ndrwxr-xr-x  6 korg  staff    192 Jan 12 23:54 dockers\n-rw-r--r--  1 korg  staff   9905 Jan 13 22:02 exrunner.ipynb\ndrwxr-xr-x  5 korg  staff    160 Jan 13 12:51 my_vol\n-rw-r--r--  1 korg  staff   3463 Dec 23 22:07 notes.txt\n-rw-r--r--  1 korg  staff    391 Dec 23 22:07 requirements.txt\n-rw-r--r--  1 korg  staff  15562 Jan 12 15:06 sender_tr.log\n-rw-r--r--  1 korg  staff   1343 Jan 12 15:06 simulation_result.json\ndrwxr-xr-x  4 korg  staff    128 Jan 13 01:01 tools\n',
 b'')

In [ ]:
try:
    os.mkdir('/mnt/nfs_share/solana/ad160')
exec:
    pass

for i in range(2):
    os.mkdir('/mnt/nfs_share/solana/ad160v' +str(i)+ '/logs')
    os.mkdir('/mnt/nfs_share/solana/ad160v' +str(i)+ '/config')
    os.mkdir('touch /mnt/nfs_share/solana/ad160v0/config/1.txt')
    